In [1]:
from __future__ import print_function # so print doesn't show brackets
import qinfer as qi
import qutip as qt
import numpy as np
import scipy as sp
#import qutip as qt
import sys as sys
import os as os

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


In [2]:
## Partial trace functionality

def expectation_value(ham, t, state=None, choose_random_probe=False):
# todo: list of probes, maybe 5 is enough? test with different values
    if choose_random_probe is True: 
        num_qubits = int(np.log2(np.shape(ham)[0]))
        state = random_probe(num_qubits)
    elif random_probe is False and state is None: 
        print ("expectation value function: you need to either pass a state or set choose_random_probe=True")
    u_psi = evolved_state(ham, t, state)
    probe_bra = state.conj().T
    psi_u_psi = np.dot(probe_bra, u_psi)
    return np.abs(psi_u_psi**2)

def evolved_state(ham, t, state):
    import hamiltonian_exponentiation as h
    unitary = h.exp_ham(ham, t)
    return np.dot(unitary, state)


def random_probe(num_qubits):
    dim = 2**num_qubits
    real = np.random.rand(1,dim)
    imaginary = np.random.rand(1,dim)
    complex_vectors = np.empty([1, dim])
    complex_vectors = real +1.j*imaginary
    norm_factor = np.linalg.norm(complex_vectors)
    probe = complex_vectors/norm_factor
    return probe[0][:]

def trim_vector(state, final_num_qubits):
#todo: renormalise
    new_vec = state[:2**final_num_qubits]/np.linalg.norm(state[:2**final_num_qubits])
    return new_vec

def qutip_evolved_state(ham, t, state):
    length = int(np.shape(ham)[0])
    evolved = evolved_state(ham,t,state=state)
    #density_mtx = np.kron(evolved.conj(), evolved).reshape(length, length)
    return qt.Qobj(evolved)

def outer_product(state, as_qutip_object=False):
    dim = int((state.shape[0]))
    if as_qutip_object:
        return qt.Qobj(np.kron(state.conj(), state).reshape(dim, dim))
    else: 
        return np.kron(state.conj(), state).reshape(dim, dim) 
 

def overlap(ham_true, ham_sim, t):
    overlap_print =False
    if overlap_print: print("overlap :")
    if overlap_print: print("ham true :" , ham_true)
    if overlap_print: print("ham sim :", ham_sim)
    if overlap_print: print("t=", t)

    true_dim = int(np.log2(np.shape(ham_true)[0])) 
    sim_dim = int(np.log2(np.shape(ham_sim)[0]))

    
    if true_dim == sim_dim:
        joined_ham = ham_sim - ham_true
        return expectation_value(joined_ham, t, choose_random_probe=True)

    min_dim = min(true_dim, sim_dim)
    max_dim = max(true_dim, sim_dim)
    to_keep = range(min_dim)

    probe = random_probe(max_dim)
    reduced_probe = trim_vector(probe, final_num_qubits=min_dim)
    
    if sim_dim > min_dim: 
    #todo: remove partial trace when system is smallest one anyway
    # if dims match -> don't go into qutip (expectation_value function); 
    # if one bigger -> ptrace on bigger Qobj on other -> get qt.expect
        sim = qutip_evolved_state(ham_sim, t, probe)
        sim.dims = [[2]*sim_dim, [1]*sim_dim]
        sim_reduced = sim.ptrace(to_keep)
    else:
        sim =  evolved_state(ham_sim, t, reduced_probe)
        #sim.dims = [[2]*sim_dim, [1]*sim_dim]
        sim_reduced = outer_product(sim, as_qutip_object=True)
    
    if true_dim > min_dim: 
        true = qutip_evolved_state(ham_true, t, probe)
        true.dims = [[2]*true_dim, [1]*true_dim]
        true_reduced = true.ptrace(to_keep)
    else: 
        true = evolved_state(ham_true, t, reduced_probe)
        #true.dims = [[2]*true_dim, [1]*true_dim]
        true_reduced = outer_product(true, as_qutip_object=True)
        
    #return true_reduced, sim_reduced
    overlap = qt.expect(sim_reduced, true_reduced)
    if overlap_print: print("overlap is ", overlap)
    return overlap

In [6]:
import qutip as qt
import hamiltonian_exponentiation as h

def iqle_evolve(ham_sim, ham_true,t):
    true_dim = int(np.log2(np.shape(ham_true)[0])) 
    sim_dim = int(np.log2(np.shape(ham_sim)[0]))

    
    if true_dim == sim_dim: 
        ham = ham_sim - ham_true
        probe = random_probe(true_dim)
        reversed_evolved_probe = expectation_value(ham, t, state=probe) 
        return reversed_evolved_probe

    elif true_dim > sim_dim:
        dim = true_dim
        smaller_dim = sim_dim
        probe = random_probe(dim)
        qt_probe = qt.Qobj(trim_vector(probe, final_num_qubits=smaller_dim))
        print("qt probe: ", qt_probe)
        to_keep = range(smaller_dim)
        evolved_probe = evolved_state(ham=ham_true, t=1, state=probe)
        evolved_qt_obj = qt.Qobj(evolved_probe)
        evolved_qt_obj.dims = [[2]*dim, [1]*dim]
        evolved_partial_trace = evolved_qt_obj.ptrace(sim_dim)[:]
        sim_unitary = h.exp_ham(ham_sim, t, plus_or_minus=1)
        sim_unitary_dagger = sim_unitary.conj().T # U_adjoint
        Rho_U = np.dot(evolved_partial_trace, sim_unitary)
        U_adjoint_Rho_U = qt.Qobj(np.dot(sim_unitary_dagger, Rho_U))
        U_adjoint_Rho_U.dims = [[2]*smaller_dim, [2]*smaller_dim]
        print("U_rho_U = ", U_adjoint_Rho_U)
        expected_value = qt.expect(U_adjoint_Rho_U, qt_probe)
        return expected_value

In [7]:
ham_sim = h.random_hamiltonian(1)
ham_true = h.random_hamiltonian(3)

In [8]:
iqle_evolve(ham_sim, ham_true, 1)

qt probe:  Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[ 0.64105256+0.63276171j]
 [ 0.03108044+0.43324155j]]
U_rho_U =  Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.61771249 +2.77555756e-17j  0.44092319 +1.34292077e-01j]
 [ 0.44092319 -1.34292077e-01j  0.38228751 +2.77555756e-17j]]


0.7633025580558671

In [216]:
u = np.array([[ 0.54253547 +0.00000000e+00j, -0.31389550 +3.69974963e-01j],
 [-0.31389550 -3.69974963e-01j,  0.45746453 +2.77555756e-17j]])

In [217]:
qt_u = qt.Qobj(u)

In [218]:
qt_u

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.54253547 +0.00000000e+00j -0.31389550 +3.69974963e-01j]
 [-0.31389550 -3.69974963e-01j  0.45746453 +2.77555756e-17j]]